In [1]:
# standard packages
from os import listdir
from tqdm import tqdm
import os
import pickle
import dill as dill_pickle

# data science stack packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import ExtraTreesRegressor

# our implementation
from dataloader import DataLoader, FeatureLoader
from features import *
from selection import GridSearchWeightedCV
from selection import SelectKBestIC, SelectKBestWeightedR2
from pipeline import Winsorizer
from EDA import EDA
from get_IC import *
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')


In [2]:
%%time

# first read the data from raw dataset
if os.path.isfile('data/data.p') == True:
    data = pickle.load(open('data/data.p', 'rb'))
else:
    dataloader = DataLoader('data')
    data = dataloader.load_features(start_date='20140102', end_date='20171229', how='outer')
    pickle.dump(data, open('data/data.p', 'wb'))

Wall time: 732 ms


In [3]:
#Maybe better to put these three features to dataloader.py

#Retrieve price from return.
#Benchmark: the price of all stocks on 20140102 being 100
price_from_return = (data['RawNoWinsorCumReturn_close']+1).cumprod()
price_from_return = 100*price_from_return/price_from_return.iloc[0,:]
data['PriceFromReturn_close'] = price_from_return

#factor 6
#Intra-day return
intraday_return = (data['CleanMid_close']-data['CleanMid_open'])/data['CleanMid_open']
data['IntradayReturn'] = intraday_return

#factor 5
#volatility weighted return
vol_weighted_return = data['ResidualNoWinsorCumReturn_close']/data['estVol']
vol_weighted_return.dropna(axis=1,how='all').shape
data['VolWeightedReturn'] = vol_weighted_return

In [4]:
list(data)

['MDV_63',
 'CleanMid_open',
 'CleanMid_close',
 'CumVolume_open',
 'CumVolume_close',
 'IsOpen_open',
 'IsOpen_close',
 'ResidualNoWinsorCumReturn_open',
 'ResidualNoWinsorCumReturn_close',
 'RawNoWinsorCumReturn_open',
 'RawNoWinsorCumReturn_close',
 'estVol',
 'SharesOutstanding',
 'PriceFromReturn_close',
 'IntradayReturn',
 'VolWeightedReturn']

# feature engineering

In [5]:
feature_loader = FeatureLoader(raw_dir='data', # the raw dataset 
                               feature_dir='features', # where to save the features
                               target_dir='data/return', # your target directory
                               data=data)

In [10]:
# first you register your feature extractor into the featureloader
# remember to pass the keyword arguments of your extractor

# Jordan's features
feature_loader.register(X_weighted_Y, 
                        name='ts_sum(res_ret*vol,20)/ts_sum(vol,20)',
                        X_name='CumVolume_close', 
                        Y_name='ResidualNoWinsorCumReturn_close', N=20)
feature_loader.register(corr_X_Y, 
                        name='corr(vol,raw_ret,20)',
                        X_name='CumVolume_close', 
                        Y_name='RawNoWinsorCumReturn_close', N=20)
feature_loader.register(ts_kurt_X, 
                        name='ts_kurt(raw_ret,20)', 
                        X_name='RawNoWinsorCumReturn_close', N=20)
feature_loader.register(ts_sum_X, 
                        name='ts_sum(res_ret,5)', 
                        X_name='ResidualNoWinsorCumReturn_close', N=20)
feature_loader.register(MQ, 
                        name='MQ(res_ret,10,0.75,0.25)', 
                        X_name='ResidualNoWinsorCumReturn_close', 
                        N=10, X_rank=False, first_quantile=0.75, second_quantile=0.25)
feature_loader.register(MQ, 
                        name='MQ(res_ret,20,0.75,0.25)', 
                        X_name='ResidualNoWinsorCumReturn_close', 
                        N=20, X_rank=False, first_quantile=0.75, second_quantile=0.25)
feature_loader.register(ts_sum_X_Y, 
                        name='ts_sum(res_ret_day,20)', 
                        X_name='ResidualNoWinsorCumReturn_close',
                        Y_name='ResidualNoWinsorCumReturn_open', 
                        N=20, __and__=np.subtract)
feature_loader.register(ts_sum_X, 
                        name='ts_sum(raw_ret,5)', 
                        X_name='RawNoWinsorCumReturn_close', N=5)
feature_loader.register(MQ, 
                        name='MQ(raw_ret,20,0.75,0.25)',
                        X_name='RawNoWinsorCumReturn_close', 
                        N=20, X_rank=False, first_quantile=0.75, second_quantile=0.25)
feature_loader.register(ts_kurt_X, 
                        name='ts_kurt(rank(estVol),20)', 
                        X_name='estVol', N=20, X_rank=True)
feature_loader.register(ts_kurt_X, 
                        name='rank(ts_kurt(vol,20))',
                        X_name='CumVolume_close', N=20, feature_rank=True)

# Kaylie's features
feature_loader.register(Squared, 
                        name='RawReturn_LogLogSquared', 
                        input_variable='RawNoWinsorCumReturn_close')
feature_loader.register(Reversal, 
                        name='Reversal1D', window=0)
feature_loader.register(Reversal, 
                        name='Reversal5D', window=4)
feature_loader.register(Liquidity, 
                        name='Liquidity_volume_close', 
                        input_variable_name='CumVolume_close', window=5)
feature_loader.register(DataNdaysBefore, 
                        name='IntradayReturn', 
                        input_variable_name='IntradayReturn', window=0)
feature_loader.register(FD, 
                        name='FD1008', 
                        input_variable='PriceFromReturn_close', window=10, d=0.8)
feature_loader.register(BB, 
                        name='BB', input_variable_name='PriceFromReturn_close', 
                        window=20,k=2)
feature_loader.register(DataNdaysBefore, 
                        name='RawNoWinsorCumReturn_close', 
                        input_variable_name='RawNoWinsorCumReturn_close', window=0)

# weight
feature_loader.register(InversedVol, name='1/estVol')

In [12]:
%%time
feature_loader.extract(start_date='20140102', end_date='20171228', save=True);

Wall time: 29min 9s


In [6]:
df_X_y = feature_loader.load_features(start_date='20140102', end_date='20171228')

list(df_X_y.columns)

['Id',
 'ts_sum(res_ret*vol,20)/ts_sum(vol,20)',
 'corr(vol,raw_ret,20)',
 'ts_kurt(raw_ret,20)',
 'ts_sum(res_ret,5)',
 'MQ(res_ret,10,0.75,0.25)',
 'MQ(res_ret,20,0.75,0.25)',
 'ts_sum(res_ret_day,20)',
 'ts_sum(raw_ret,5)',
 'MQ(raw_ret,20,0.75,0.25)',
 'ts_kurt(rank(estVol),20)',
 'rank(ts_kurt(vol,20))',
 'RawReturn_LogLogSquared',
 'Reversal1D',
 'Reversal5D',
 'Liquidity_volume_close',
 'IntradayReturn',
 'FD1008',
 'BB',
 'RawNoWinsorCumReturn_close',
 '1/estVol',
 'ResidualNoWinsorCumReturn',
 'Date']

# ML models

In [7]:
train_years = {2014, 2015, 2016}
test_years = {2017}

Jordan_flag = True
Kaylie_flag = True

X_y_dropna = False    # if True, dropna in X and y
y_min = -0.05    # min values to clip y 
y_max = 0.05    # max values to clip y

select_by_IC = False
select_by_R2 = False
num_features = 10    # number of features to select

In [8]:
# get X and y
Jordan_features = [
    'ts_sum(res_ret*vol,20)/ts_sum(vol,20)',   
    'corr(vol,raw_ret,20)',
    'ts_kurt(raw_ret,20)', 
    'ts_sum(res_ret,5)', 
    'MQ(res_ret,10,0.75,0.25)',
    'MQ(res_ret,20,0.75,0.25)',  
    'ts_sum(res_ret_day,20)',      
    'ts_sum(raw_ret,5)',                 
    'MQ(raw_ret,20,0.75,0.25)',               
    'ts_kurt(rank(estVol),20)',            
    'rank(ts_kurt(vol,20))',                   
]
Kaylie_features = [
    'RawReturn_LogLogSquared',
    'Reversal1D',
    'Reversal5D',                   
    'Liquidity_volume_close',       
    'IntradayReturn',                         
    'FD1008',                          
    'BB',                      
    'RawNoWinsorCumReturn_close',      
]

X_y_features = []
if Jordan_flag is True:
    X_y_features += Jordan_features 
if Kaylie_flag is True:
    X_y_features += Kaylie_features 
X_y = df_X_y[X_y_features + ['ResidualNoWinsorCumReturn', '1/estVol', 'Date', 'Id']]


if X_y_dropna == True:
    X_y.dropna(inplace=True)
X_y.index = pd.to_datetime(X_y['Date'])

In [9]:
# train test split
X_y_train = X_y[X_y.index.year.isin(train_years)]
X_y_test = X_y[X_y.index.year.isin(test_years)]

X_train = X_y_train.iloc[:,:-4]
y_train = X_y_train['ResidualNoWinsorCumReturn']
w_train = X_y_train['1/estVol']

X_test = X_y_test.iloc[:,:-4]
y_test = X_y_test['ResidualNoWinsorCumReturn']
w_test = X_y_test['1/estVol']

## XGBoost, with GridSearchWeightedCV

In [10]:
%%time

pipe_xgb = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                           ('winsorizer', Winsorizer(qmin=0.01, qmax=0.99)),
                           ('scaler', StandardScaler()),
#                            ('selector', SelectKBestIC(verbose=False, k=num_features)),
#                            ('R2selector', SelectKBestWeightedR2(verbose=True, k=num_features)),
                           ('xgb', xgb.XGBRegressor(n_jobs=5, random_state=42))
                          ])

# clip y, in pipeline we have to separate the preprocess of X and y
# in case of how to clip, I suggest we do an explortory data analysis
# here I use 0.01 and 0.99 quantile (not the exact number but just the same scale)
# since we used the whole training dataset to get this result
# when we do cross-validation we will peek the future
tt_xgb = TransformedTargetRegressor(regressor=pipe_xgb,
                                    func=lambda y: np.clip(y, a_min=y_min, a_max=y_max),
                                    inverse_func=lambda y: y,
                                    check_inverse=False)

# we have some fit parameters to pass in this model
# you don't need xgb__ prefix for the keyword here
fit_params = {
    "early_stopping_rounds": 5,
    'verbose': 1
}
# some parameters to tune
param_grid = {    
    'xgb__learning_rate': [0.1],
    'xgb__n_estimators': [300],
    'xgb__max_depth': [3],
    'xgb__reg_alpha': [1],
    'xgb__subsample': [0.5],
    'xgb__colsample_bytree': [0.5],
}

search_xgb = GridSearchWeightedCV(estimator=tt_xgb,
                                  param_grid=param_grid,
                                  eval_set=True,
                                  cv=5,
                                  verbose=True,
                                  random_state=42)
search_xgb.fit(X_train, y_train,
               Date=X_y_train['Date'], 
               Id=X_y_train['Id'],
               sample_weight=w_train.fillna(value=0),
               **fit_params)

print('weighted R^2 in train set: ', search_xgb.best_score_)

{'xgb__colsample_bytree': 0.5, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 300, 'xgb__reg_alpha': 1, 'xgb__subsample': 0.5}
[0]	validation_0-rmse:0.45015
[1]	validation_0-rmse:0.40517
[2]	validation_0-rmse:0.36469
[3]	validation_0-rmse:0.32827
[4]	validation_0-rmse:0.29548
[5]	validation_0-rmse:0.26599
[6]	validation_0-rmse:0.23945
[7]	validation_0-rmse:0.21557
[8]	validation_0-rmse:0.19408
[9]	validation_0-rmse:0.17475
[10]	validation_0-rmse:0.15736
[11]	validation_0-rmse:0.14172
[12]	validation_0-rmse:0.12766
[13]	validation_0-rmse:0.11501
[14]	validation_0-rmse:0.10365
[15]	validation_0-rmse:0.09343
[16]	validation_0-rmse:0.08425
[17]	validation_0-rmse:0.07600
[18]	validation_0-rmse:0.06860
[19]	validation_0-rmse:0.06197
[20]	validation_0-rmse:0.05602
[21]	validation_0-rmse:0.05069
[22]	validation_0-rmse:0.04592
[23]	validation_0-rmse:0.04166
[24]	validation_0-rmse:0.03785
[25]	validation_0-rmse:0.03447
[26]	validation_0-rmse:0.03146
[27]	validation_0-rmse:0

In [11]:
print(search_xgb.best_params_)
print('weighted R^2 in train set: ', search_xgb.best_estimator.score(X_train, y_train, sample_weight=w_train))
print('weighted R^2 in test set: ', search_xgb.best_estimator.score(X_test, y_test, sample_weight=w_test))

{'xgb__subsample': 0.5, 'xgb__reg_alpha': 1, 'xgb__n_estimators': 300, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.1, 'xgb__colsample_bytree': 0.5}
weighted R^2 in train set:  0.0016761314401378957
weighted R^2 in test set:  0.0005654943239634669


## XGBoost, with GridSearchWeightedCV

In [25]:
# # load model
# with open('models/xgboost, 19 features.pickle', 'rb') as f:
#     tt_xgb = dill_pickle.load(f)

# print('weighted R^2 in train set: ', tt_xgb.score(X_train, y_train, sample_weight=w_train))
# print('weighted R^2 in test set: ', tt_xgb.score(X_test, y_test, sample_weight=w_test))

weighted R^2 in train set:  0.00120500092863296
weighted R^2 in test set:  0.0003740739962669881


In [ ]:
# # predict
# y_pred_train_xgb = pd.DataFrame(tt_xgb.predict(X_train), index=y_train.index, columns=['y_pred'])
# y_pred_test_xgb = pd.DataFrame(tt_xgb.predict(X_test), index=y_test.index, columns=['y_pred'])
# y_pred_xgb = pd.concat([y_pred_train_xgb, y_pred_test_xgb])
# y_pred_xgb.to_pickle('models/y_pred_xgb, 19 features.pickle')